part 5 : loading a new datset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
from google.colab import files

# Upload the .zip file
uploaded = files.upload()

# Extract the contents of the .zip file
import zipfile
import io

with zipfile.ZipFile("/content/drive/MyDrive/brain_tumor_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall('/content/dataset3')
    zip_ref.close()

Saving brain_tumor_dataset.zip to brain_tumor_dataset (1).zip


part 6 : finetuning VITMAE with the new dataset

In [ ]:
import os
import pandas as pd
from PIL import Image
import csv
from torch.utils.data import Dataset
from transformers import ViTFeatureExtractor, TrainingArguments, Trainer, ViTMAEForPreTraining

In [ ]:
import zipfile
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from transformers import ViTFeatureExtractor, ViTMAEForPreTraining, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
from PIL import Image
import numpy as np

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, image_dir , feature_extractor):
        self.image_dir = image_dir
        self.feature_extractor = feature_extractor
        # Get a list of all image files in the directory
        self.image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        inputs = self.feature_extractor(images=image, return_tensors="pt")

        # Extract the corresponding label from the CSV
        image_file_name = os.path.basename(img_path)

        return {
            'pixel_values': inputs['pixel_values'].squeeze(),
            'noise': None,  # Add other keys as necessary, using None or default values if not applicable
            'head_mask': None,
            'output_attentions': None,
            'output_hidden_states': None,
            'return_dict': None,
            'interpolate_pos_encoding': None,
            'label_ids': None,
            'labels': None  # Return the label along with the pixel values
        }

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained("facebook/vit-mae-base")

preprocessor_config.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os

In [ ]:
image_dir_train = '/content/dataset3/brain_tumor_dataset/test_dataset'
image_dir_test = '/content/dataset3/brain_tumor_dataset/train_dataset'

In [ ]:
train_dataset = CustomImageDataset(image_dir=image_dir_train,  feature_extractor=feature_extractor)
test_dataset = CustomImageDataset(image_dir=image_dir_test,  feature_extractor=feature_extractor)

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset3/brain_tumor_dataset/test_dataset'

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    evaluation_strategy="epoch",
    logging_dir='./logs',
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model = ViTMAEForPreTraining.from_pretrained("facebook/vit-mae-base")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/448M [00:00<?, ?B/s]

NameError: name 'training_args' is not defined

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,No log,No log
4,No log,No log
5,No log,No log


TrainOutput(global_step=155, training_loss=0.17275587512600807, metrics={'train_runtime': 75.9864, 'train_samples_per_second': 10.199, 'train_steps_per_second': 2.04, 'total_flos': 7.8330474528768e+16, 'train_loss': 0.17275587512600807, 'epoch': 5.0})

In [ ]:
trainer.save_model("./finetuned-vit-mae")

NameError: name 'trainer' is not defined

part 6 : changed